In [ ]:
from math import tau

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.linalg as LA
import scipy.signal as ss

plt.style.use('seaborn-notebook')
np.random.seed(293710966)

# Generazione del Segnale

In [ ]:
magnitudes = pd.read_csv('./harmonic-voltage-magnitude.csv', index_col='number')['typical']
phases = np.radians(pd.read_csv('./phases.csv', index_col='number'))['3.9']

power_freq = 50
sampling_freq = 2400
harmonic_numbers = np.arange(2, 14)
no_of_harmonics = harmonic_numbers.max() * 2

time = np.arange(4096.)

noise = np.random.normal(0, 0.0001, time.size)
signal = noise.copy()

for n in harmonic_numbers:
    amp = magnitudes[n]
    phase = phases[n]
    omega = tau * n * power_freq / sampling_freq
    signal += amp * np.cos(omega * time + phase)

fig, ax = plt.subplots()
ax.plot(time[:200], signal[:200])
ax.set_xlabel('time [s]')
ax.set_ylabel('signal [V]')
fig.tight_layout()

In [ ]:
fastest_period = np.around(sampling_freq / power_freq).astype(int)
time_window = fastest_period * harmonic_numbers[-1]
data_size = signal.size - time_window + 1

windows = [signal[i : i + time_window] for i in range(data_size)]
data_matrix = np.vstack(windows)

In [ ]:
correlation = data_matrix.conj().T @ data_matrix / data_matrix.shape[0]

In [ ]:
# signal: K largest      -> M - K : M - 1
# noise: M - K smallest  -> 0 : M - K - 1

signal_pca = LA.eigh(correlation, subset_by_index=(time_window - no_of_harmonics, time_window - 1))
noise_pca = LA.eigh(correlation, subset_by_index=(0, time_window - no_of_harmonics - 1))

In [ ]:
omegas = np.linspace(0, np.pi, 2000)
freqs = omegas * sampling_freq / tau

pseudo_power = np.empty_like(omegas)

for i, omega in enumerate(omegas):
    steering = np.exp(1j * omega * np.arange(time_window))
    pseudo_power[i] = 1 / np.sum(np.abs(noise_pca[1].T @ steering.conj()) ** 2)
    
fig, ax = plt.subplots()
ax.plot(freqs, 20 * np.log10(pseudo_power))
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Pseudospectrum [dB]')
fig.tight_layout()

In [ ]:
peaks_idx, _ = ss.find_peaks(pseudo_power)

peaks = pd.DataFrame()
peaks['omega'] = omegas[peaks_idx]
peaks['freq'] = freqs[peaks_idx]
peaks['power'] = pseudo_power[peaks_idx]
peaks = peaks.sort_values('power').tail(harmonic_numbers.size).sort_values('omega')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(freqs, pseudo_power)
ax1.plot(peaks.freq, peaks.power, 'o');
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('Pseudospectrum')
ax1.set_yscale('log')

ax2.plot(power_freq * harmonic_numbers, 'o-', label='Harmonic Frequencies')
ax2.plot(peaks.freq.values, 'o-', label='Estimated Frequencies')
ax2.set_ylabel('Frequency [Hz]')
ax2.legend()

fig.tight_layout()